In [1]:
from __future__ import print_function
import requests

In [4]:
def get_ans():
	'''Asks user for input. If the user types yes true is returned'''
	ans = input()
	if ans.lower == 'y' or ans.lower == 'yes' or ans == '1':
		return True
	else:
		return False

def get_wcif(comp):
	'''Pulls WCIF from WCA and return json object. If there is a problem with the request the program terminates'''
	print('Attempting to pull competition info...')
	# Send GET request for WCIF
	WCIF = requests.get('https://www.worldcubeassociation.org/api/v0/competitions/' + comp + '/wcif/public')
	# If response code is 200 everything good, otherwise something bad
	if WCIF.status_code == 200:
		print('Success!')
		return WCIF.json()
	else:
		print('Response Error!')
		exit()

In [5]:
# Get comp name from user
print('Enter competition ID:')
#comp = input()
comp = 'Cubinginthe6ix2019'
#comp_file = comp + '-registration.csv'

WCIF = get_wcif(comp)

Enter competition ID:
Attempting to pull competition info...
Success!


In [74]:
class Competitor:
    def __init__(self, name, ID, events, wcaid):
        self.name = name
        self.id = ID
        self.events = events
        self.wcaid = wcaid
        
    @staticmethod
    def build_competitors(WCIF):
        lst = [None] * len(WCIF['persons'])
        i = 0
        for pers in WCIF['persons']:
            lst[i] = Competitor(pers['name'], pers['registrantId'], pers['registration']['eventIds'], pers['wcaId'])
            i += 1
        return lst
    


In [75]:
c1 = Competitor('Abdullah Gulab', 2, ['333', '333bf', '444', '666', '777'], '2014GULA02')
c2 = Competitor.build_competitors(WCIF)

In [76]:
print(c2[50].events)

[u'333', u'444']
